In [ ]:
!git clone https://github.com/geohot/tinygrad.git

Cloning into 'tinygrad'...
remote: Enumerating objects: 14649, done.
remote: Counting objects: 100% (2046/2046), done.
remote: Compressing objects: 100% (454/454), done.
remote: Total 14649 (delta 1813), reused 1695 (delta 1591), pack-reused 12603
Receiving objects: 100% (14649/14649), 18.45 MiB | 23.13 MiB/s, done.
Resolving deltas: 100% (10342/10342), done.


In [ ]:
cd tinygrad/

/content/tinygrad


In [ ]:
!python3 -m pip install -e .

Obtaining file:///content/tinygrad
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.2/919.2 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 7.7 MB/s eta 0:00:00
  Running setup.py develop for tinygrad


In [ ]:
import time, sys
import json
import numpy as np
from tinygrad.tensor import Tensor
from tinygrad.nn import optim
import tinygrad.nn as nn
from tinygrad.helpers import flatten
from tinygrad.nn.optim import SGD, Adam
from sklearn.datasets import fetch_openml
from tinygrad.state import safe_save, safe_load, get_state_dict, load_state_dict, torch_load

In [ ]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False, )
print(X.shape, y.shape)

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


(70000, 784) (70000,)


In [ ]:
class LeNetTinyNet:
    def __init__(self):
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5)
        self.fc1 = nn.Linear(in_features=120, out_features=84)
        self.fc2 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        x = x.reshape((x.shape[0], 1, 28, 28))
        x = self.conv1(x)
        x = x.tanh()
        x = x.avg_pool2d(2)    ## Conv1:  (64, 6, 14, 14)

        x = self.conv2(x)
        x = x.tanh()
        x = x.avg_pool2d(2)    ## Conv2:  (64, 16, 5, 5)

        x = self.conv3(x)
        x = x.tanh()

        x = x.reshape((x.shape[0], -1))  ## Reshape:  (64, 120)

        x = self.fc1(x)  ## FC1:  (64, 84)
        x = x.tanh()

        x = self.fc2(x)   ## FC2:  (64, 10)
        return x.log_softmax()

net = LeNetTinyNet()

In [ ]:
Tensor.training = True

In [ ]:
# from extra.training import sparse_categorical_crossentropy
def cross_entropy(out, Y):
  num_classes = out.shape[-1]
  YY = Y.flatten().astype(np.int32)
  y = np.zeros((YY.shape[0], num_classes), np.float32)
  y[range(y.shape[0]),YY] = -1.0*num_classes
  y = y.reshape(list(Y.shape)+[num_classes])
  y = Tensor(y)
  return out.mul(y).mean()

In [ ]:
opt = Adam([net.conv1.weight, net.conv2.weight, net.conv3.weight], lr=3e-4)

In [ ]:
num_epochs = 2000

weight_bias_dictionary = {}
running_loss, correct, total = 0.0, 0.0, 0.0
for epoch in range(num_epochs):
    weigth_bias = {}
    start_time = time.time()

    # Randomly sample a batch
    samp = np.random.randint(0, X.shape[0], size=(64))
    batch = Tensor(X[samp].astype('float32') / 255.0, requires_grad=False)
    # Get the corresponding labels
    labels = y[samp]

    # Forward pass
    out = net.forward(batch)

    # Compute loss
    loss = cross_entropy(out, labels)

    # Zero gradients
    opt.zero_grad()

    # Backward pass
    loss.backward()

    # Update parameters
    opt.step()

    # Calculate accuracy
    pred = np.argmax(out.numpy(), axis=-1)
    labels = [eval(label) for label in labels]

    acc = (pred == labels).mean()
    if epoch % 100 == 0:
      print(f"Time Taken: {time.time()-start_time:.3f}s, Epoch [{epoch+1}/{num_epochs}], Loss: {loss.numpy():.5f}, Accuracy: {acc:.5f}")

Time Taken: 0.440s, Epoch [1/2000], Loss: 2.29542, Accuracy: 0.14062
Time Taken: 0.379s, Epoch [101/2000], Loss: 1.58329, Accuracy: 0.78125
Time Taken: 0.245s, Epoch [201/2000], Loss: 1.29733, Accuracy: 0.89062
Time Taken: 0.246s, Epoch [301/2000], Loss: 1.20763, Accuracy: 0.87500
Time Taken: 0.243s, Epoch [401/2000], Loss: 1.23674, Accuracy: 0.82812
Time Taken: 0.230s, Epoch [501/2000], Loss: 1.07521, Accuracy: 0.92188
Time Taken: 0.249s, Epoch [601/2000], Loss: 1.08164, Accuracy: 0.92188
Time Taken: 0.225s, Epoch [701/2000], Loss: 1.09325, Accuracy: 0.87500
Time Taken: 0.246s, Epoch [801/2000], Loss: 1.06176, Accuracy: 0.89062
Time Taken: 0.247s, Epoch [901/2000], Loss: 1.04841, Accuracy: 0.89062
Time Taken: 0.240s, Epoch [1001/2000], Loss: 1.04766, Accuracy: 0.90625
Time Taken: 0.246s, Epoch [1101/2000], Loss: 1.03908, Accuracy: 0.89062
Time Taken: 0.388s, Epoch [1201/2000], Loss: 1.01787, Accuracy: 0.90625
Time Taken: 0.394s, Epoch [1301/2000], Loss: 1.00754, Accuracy: 0.90625
Time

In [ ]:
# set training flag to false
Tensor.training = False

st = time.perf_counter()
avg_acc = 0
for step in range(1000):
  # random sample a batch
  samp = np.random.randint(0, X.shape[0], size=(64))
  batch = Tensor((X[samp].astype('float32') / 255.0), requires_grad=False)
  # get the corresponding labels
  labels = y[samp]

  # forward pass
  out = net.forward(batch)

  # calculate accuracy
  pred = np.argmax(out.numpy(), axis=-1)

  labels = [eval(label) for label in labels]
  avg_acc += (pred == labels).mean()

print(f"Test Accuracy: {avg_acc / 1000}")
print(f"Time Taken To Test: {time.perf_counter() - st}")

Test Accuracy: 0.931875
Time Taken To Test: 81.81800240500002


In [ ]:
# first we need the state dict of our model
state_dict = get_state_dict(net)

# then we can just save it to a file
safe_save(state_dict, "/content/model.safetensors")

In [ ]:
!wget https://i.stack.imgur.com/VChE0.png

--2023-07-06 06:56:37--  https://i.stack.imgur.com/VChE0.png
Resolving i.stack.imgur.com (i.stack.imgur.com)... 146.75.28.193
Connecting to i.stack.imgur.com (i.stack.imgur.com)|146.75.28.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 307274 (300K) [image/png]
Saving to: ‘VChE0.png’

VChE0.png           100%[===================>] 300.07K  --.-KB/s    in 0.04s   

2023-07-06 06:56:37 (7.98 MB/s) - ‘VChE0.png’ saved [307274/307274]



In [ ]:
import tinygrad.tensor as tensor
from PIL import Image
# Step 1: Preprocess the image
# Your preprocessing code here
image = Image.open('/content/tinygrad/VChE0.png')
image = image.resize((28, 28))  # Resize the image to match the input size of the model

# Step 2: Load the trained model
state_dict = safe_load("/content/model.safetensors")

checkpoints = {
    'conv1.weight': state_dict['conv1.weight'],
    'conv1.bias': state_dict['conv1.bias'],
    'conv2.weight': state_dict['conv2.weight'],
    'conv2.bias': state_dict['conv2.bias'],
    'conv3.weight': state_dict['conv3.weight'],
    'conv3.bias': state_dict['conv3.bias'],
    'fc1.weight': state_dict['fc1.weight'],
    'fc1.bias': state_dict['fc1.bias'],
    'fc2.weight': state_dict['fc2.weight'],
    'fc2.bias': state_dict['fc2.bias']
}

model = LeNetTinyNet()

load_state_dict(model, checkpoints)

image = image.convert("L")

image = np.array(image)

image = image.astype(np.float32) / 255.0  # Normalize pixel values to the range [0, 1]

start_time = time.time()
preprocessed_image = image.reshape((1, 1, 28, 28))

# Step 3: Forward pass
input_tensor = tensor.Tensor(preprocessed_image)  # Create a tensor from the preprocessed image
output_tensor = net.forward(input_tensor)  # Perform forward pass

print(f"\n\nTime Taken to Predict the class: {time.time() - start_time: .4f}")

# Step 4: Obtain predictions
predicted_class= np.argmax(output_tensor.numpy(), axis=-1)
print("Predicted class:", predicted_class)

ram used:  0.30 GB, fc2.bias                                          : 100%|██████████| 10/10 [00:00<00:00, 469.47it/s]

loaded weights in 29.61 ms, 0.30 GB loaded at 10.15 GB/s


Time Taken to Predict the class:  0.0295
Predicted class: [2]
